### Predicting Home Prices in Bangalore

Dataset is downloaded from here: https://www.kaggle.com/amitabhajoy/bengaluru-house-price-data

In [45]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [46]:
df = pd.read_csv("data\Bengaluru_House_Data.csv")
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [47]:
df.shape

(13320, 9)

In [48]:
df.columns

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony', 'price'],
      dtype='object')

In [49]:
df['area_type'].unique()

array(['Super built-up  Area', 'Plot  Area', 'Built-up  Area',
       'Carpet  Area'], dtype=object)

In [50]:
df['area_type'].value_counts()

Super built-up  Area    8790
Built-up  Area          2418
Plot  Area              2025
Carpet  Area              87
Name: area_type, dtype: int64

Let's consider *location, size, total_sqft, bath and price* as our primary features

In [51]:
condensed_df = df.drop(['area_type','society','balcony'
                        ,'availability'],axis='columns')
condensed_df.columns

Index(['location', 'size', 'total_sqft', 'bath', 'price'], dtype='object')

Let's clean the data by removing the NA values

In [52]:
condensed_df.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [53]:
condensed_df.shape

(13320, 5)

In [54]:
clean_df = condensed_df.dropna()
clean_df.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

In [55]:
clean_df.shape

(13246, 5)

We lost 74 records after removing the na values. It is acceptable as the *lost* record count is very less.

#### Feature Engineering

In [56]:
clean_df["size"].value_counts()

2 BHK         5198
3 BHK         4286
4 Bedroom      818
4 BHK          577
3 Bedroom      546
1 BHK          531
2 Bedroom      329
5 Bedroom      296
6 Bedroom      191
1 Bedroom      105
8 Bedroom       84
7 Bedroom       83
5 BHK           57
9 Bedroom       46
6 BHK           30
7 BHK           17
1 RK            13
10 Bedroom      12
9 BHK            8
8 BHK            5
10 BHK           2
11 Bedroom       2
11 BHK           2
14 BHK           1
18 Bedroom       1
27 BHK           1
43 Bedroom       1
13 BHK           1
12 Bedroom       1
16 BHK           1
19 BHK           1
Name: size, dtype: int64

In [57]:
clean_df["bhk"] = clean_df["size"].apply(lambda x: int(x.split(' ')[0]))

In [58]:
clean_df["bhk"].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

In [59]:
clean_df.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [60]:
clean_df[clean_df.bhk > 25]

,location,size,total_sqft,bath,price,bhk
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43


In [61]:
clean_df.total_sqft.unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [62]:
def is_float(f):#Check if a field is float or not ?
    try:
        float(f)
    except:
        return False
    return True

In [63]:
clean_df[~clean_df["total_sqft"].apply(is_float)].shape

(190, 6)

In [64]:
clean_df[~clean_df["total_sqft"].apply(is_float)].head(10)

,location,size,total_sqft,bath,price,bhk
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2
410,Kengeri,1 BHK,34.46Sq. Meter,1.0,18.500,1
549,Hennur Road,2 BHK,1195 - 1440,2.0,63.770,2
648,Arekere,9 Bedroom,4125Perch,9.0,265.000,9
661,Yelahanka,2 BHK,1120 - 1145,2.0,48.130,2
672,Bettahalsoor,4 Bedroom,3090 - 5002,4.0,445.000,4


In [65]:
#Convert the Square foot Range into a number(by taking an average)
def convert_sft_rng_to_num(x):
    tokens = x.split('-')
    if(len(tokens) == 2):
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return "Not a valid Square foot Range"

In [66]:
convert_sft_rng_to_num('1145 - 1340')

1242.5

In [67]:
convert_sft_rng_to_num('4125Perch')

'Not a valid Square foot Range'

In [69]:
clean_df1 = clean_df.copy()
clean_df1['total_sft'] = clean_df1['total_sqft'].apply(convert_sft_rng_to_num)
clean_df1.head()

,location,size,total_sqft,bath,price,bhk,total_sft
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2,1056
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4,2600
2,Uttarahalli,3 BHK,1440,2.0,62.00,3,1440
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3,1521
4,Kothanur,2 BHK,1200,2.0,51.00,2,1200


In [70]:
clean_df2 = clean_df1.copy()
clean_df2['price_per_sft'] = clean_df2['price']*100000/clean_df2['total_sft']
clean_df2.head()

TypeError: unsupported operand type(s) for /: 'float' and 'str'

In [ ]:
len(clean_df2.location.unique())

In [ ]:
clean_df2.location = clean_df2.location.apply(lambda x: x.strip())

In [ ]:
loc_counts = clean_df2.groupby('location')['location'].agg('count').sort_values(ascending = False)
loc_counts

In [ ]:
loc_counts_less_than_10 = loc_counts[loc_counts<=10]
loc_counts_less_than_10

In [ ]:
clean_df2.location = clean_df2.location.apply(lambda x: 'other' if x in loc_counts_less_than_10 else x)

In [ ]:
len(clean_df2.location.unique())

In [ ]:
clean_df2.head(10)

In [ ]:
clean_df2[clean_df2.total_sft/clean_df2.bhk < 300].head()

In [ ]:
clean_df2 = clean_df2.drop(['total_sqft'],axis=1)

In [ ]:
clean_df2.head()

In [ ]:
clean_df2.shape

In [ ]:
clean_df2 = clean_df2[~(clean_df2.total_sft/clean_df2.bhk < 300)]

In [ ]:
clean_df2.shape

In [ ]:
clean_df2.price_per_sft.describe()

In [ ]:
def rem_pps_outliers(df):
    out = pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sft)
        sd = np.std(subdf.price_per_sft)
        red_df = subdf[(subdf.price_per_sft>(m-sd))&(subdf.price_per_sft<=(m+sd))]
        out = pd.concat([out,red_df], ignore_index = True)
    return out
        

In [ ]:
clean_df3 = rem_pps_outliers(clean_df2)
clean_df3.shape

In [ ]:
def visualize_scatter(df,location):
    bhk2 = df[(df.location==location) & (df.bhk==2)]
    bhk3 = df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize'] = (15,8)
    sns.scatterplot(x = 'total_sft',y = 'price' ,color='blue',label='2 BHK', s=50, data = bhk2)
    sns.scatterplot(x = 'total_sft',y = 'price' ,color='orange',label='3 BHK', s=50, data = bhk3)
    plt.title(location)
    
visualize_scatter(clean_df3,"Uttarahalli")

In [ ]:
visualize_scatter(clean_df3,"Marathahalli")

In [ ]:
def rem_outliers_by_bhk(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sft),
                'std': np.std(bhk_df.price_per_sft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

In [ ]:
clean_df4 = rem_outliers_by_bhk(clean_df3)

clean_df4.shape

In [ ]:
visualize_scatter(clean_df3,"Uttarahalli")

In [ ]:
visualize_scatter(clean_df4,"Marathahalli")

In [ ]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
plt.hist(clean_df4.price_per_sft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

In [ ]:
clean_df4.bath.unique()

In [ ]:
plt.hist(clean_df4.bath,rwidth=0.8)
plt.xlabel("Number of bathrooms")
plt.ylabel("Count")

In [ ]:
clean_df4[clean_df4.bath>10]

In [ ]:
clean_df4[clean_df4.bath>clean_df4.bhk+2]

In [ ]:
clean_df5 = clean_df4[clean_df4.bath<clean_df4.bhk+2]
clean_df5.shape

In [ ]:
clean_df5.head(2)


In [ ]:
clean_df6 = clean_df5.drop(['size','price_per_sft'],axis = 'columns')

In [ ]:
clean_df6.head()

# One Hot Encoding

In [ ]:
oh = pd.get_dummies(clean_df6.location)
oh.head()

In [ ]:
clean_df7 = pd.concat([clean_df6,oh.drop('other',axis = 'columns')], axis = 'columns')
clean_df7.head()

In [ ]:
clean_df8 = clean_df7.drop('location',axis = 'columns')

In [ ]:
clean_df8.head()

In [ ]:
clean_df8.columns

In [ ]:
clean_df8.shape

In [ ]:
X = clean_df8.drop('price',axis = 'columns')
X.head()

In [ ]:
y = clean_df8.price
y.head()

## Train test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
linear_model = LinearRegression()
linear_model.fit(X_train,y_train)
print("Linear model Score : ", linear_model.score(X_test,y_test)*100)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

lin_cv = ShuffleSplit(n_splits=5,test_size=0.2,random_state=10)

cross_val_score(LinearRegression(),X,y,cv=lin_cv)

In [ ]:
#Gridsearch CV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

In [ ]:
def best_model_using_GridSearchCV(X,y):
    algos = {
                'linear_regression':{
                    'model': LinearRegression(),
                    'params': {
                        'normalize' : [True,False]
                    }
                },
                'lasso' : {
                  'model': Lasso(),
                    'params' : {
                        'alpha' :[1,2],
                        'selection' : ['random','cyclic']
                    }
                },
                'decision_tree':{
                    'model' : DecisionTreeRegressor(),
                    'params' :{
                        'criterion' : ['mse','friedman_mse'],
                        'splitter':['best','random']
                    }
                }
            }
    scores = []
    cv = ShuffleSplit(n_splits = 5, test_size=0.2,random_state = 0)
    for algo_name,config in algos.items():
        gs = GridSearchCV(config['model'],config['params'],cv=cv,return_train_score = False)
        gs.fit(X,y)
        scores.append({
            'model' : algo_name,
            'best_score' : gs.best_score_,
            'best_params' : gs.best_params_
        })
    return pd.DataFrame(scores,columns = ['model','best_score','best_params'])

In [ ]:
best_model_using_GridSearchCV(X,y)

In [ ]:
def pred_price(location,sft,bath,bhk):
    loc_index = np.where(X.columns == location)[0][0]
    
    x= np.zeros(len(X.columns))
    x[0] = bath
    x[1] = bhk
    x[2] = sft
    if(loc_index >= 0):
        x[loc_index] =1
    return linear_model.predict([x])[0]

In [ ]:
X.columns

In [ ]:
pred_price('1st Phase JP Nagar',1000,2,2)

In [ ]:
pred_price('1st Phase JP Nagar',1000,2,3)

In [ ]:
pred_price('5th Block Hbr Layout',1000,2,2)

In [ ]:
pred_price('5th Block Hbr Layout',1000,3,2)

In [ ]:
pred_price('Rajaji Nagar',1000,2,2)

In [ ]:
pred_price('Rajaji Nagar',1000,3,3)

In [ ]:
import pickle
with open('blr_home_price_model.pickle','wb') as f:
    pickle.dump(linear_model,f)

In [ ]:
import json
columns = {
    'data_columns': [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))